In [1]:
import keras
import wandb
from wandb.keras import WandbCallback
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
import DatasetParser
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.
wandb: WARNING Keras version 2.3.1 is not fully supported. Required keras >= 2.4.0


In [2]:
(x_train, y_train_raw), (x_test, y_test_raw), class_names = DatasetParser.load_data("Resnet")

224
Entrenamiento: 3348
Test: 1440


In [3]:
# One hot encode ouput
y_train = keras.utils.to_categorical(y_train_raw)
y_test = keras.utils.to_categorical(y_test_raw)

In [4]:
# Load ResNet50 Trained on imagenet
resnet_model = ResNet50(weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=True)


In [5]:
from keras.preprocessing import image
import numpy as np
img = image.load_img('elephant.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = resnet_model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n01871265', 'tusker', 0.48542696), ('n02504013', 'Indian_elephant', 0.3432928), ('n02504458', 'African_elephant', 0.17125095)]


In [6]:
# We should preprocess the images the same way resnet images were preprocessed
x_train_preprocessed = preprocess_input(x_train)
x_test_preprocessed = preprocess_input(x_test)

In [7]:
# Build a new model that is ResNet50 minus the very last layer
last_layer = resnet_model.get_layer("avg_pool")

resnet_layers = keras.Model(inputs=resnet_model.inputs, outputs=last_layer.output)
resnet_layers.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [8]:
# We use our resnet to "predict" but because we have removed the top layer, 
# this outputs the activations of the second to last layer on our dataset

x_train_features = resnet_layers.predict(x_train_preprocessed)

In [9]:
x_test_features = resnet_layers.predict(x_test_preprocessed)

In [10]:
# We can directly stich the models together

Fine_Tuning_model=Sequential()
Fine_Tuning_model.add(resnet_layers)
Fine_Tuning_model.add(Dense(36, activation="softmax"))

Fine_Tuning_model.layers[0].trainable=False

Fine_Tuning_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Fine_Tuning_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 36)                73764     
Total params: 23,661,476
Trainable params: 73,764
Non-trainable params: 23,587,712
_________________________________________________________________


In [11]:
wandb.init(project='test', entity='scueroinc')
Fine_Tuning_model.fit(x_train_preprocessed, y_train, epochs=10, validation_data=(x_test_preprocessed, y_test), callbacks=[WandbCallback()])

wandb: Currently logged in as: scueroinc (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.25 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Train on 3348 samples, validate on 1440 samples
Epoch 1/10
3348/3348 [==============================] - 46s 14ms/step - loss: 1.7471 - accuracy: 0.5579 - val_loss: 1.1332 - val_accuracy: 0.6583
Epoch 2/10
3348/3348 [==============================] - 44s 13ms/step - loss: 0.5653 - accuracy: 0.8665 - val_loss: 0.9083 - val_accuracy: 0.7389
Epoch 3/10
3348/3348 [==============================] - 44s 13ms/step - loss: 0.3322 - accuracy: 0.9295 - val_loss: 1.0060 - val_accuracy: 0.7139
Epoch 4/10
3348/3348 [==============================] - 44s 13ms/step - loss: 0.2427 - accuracy: 0.9519 - val_loss: 0.9703 - val_accuracy: 0.7278
Epoch 5/10
3348/3348 [==============================] - 44s 13ms/step - loss: 0.1775 - accuracy: 0.9704 - val_loss: 0.8623 - val_accuracy: 0.7632
Epoch 6/10
3348/3348 [==============================] - 44s 13ms/step - loss: 0.1355 - accuracy: 0.9797 - val_loss: 1.0046 - val_accuracy: 0.7285
Epoch 7/10
3348/3348 [==============================] - 44s 13ms/step - loss

In [12]:
# We can allow some of the resnet layers to change as we train.  
# Typically you would want to lower the learning rate in conjunction with this.

Fine_Tuning_model.layers[0].trainable = True

# We let the last 3 blocks train
for layer in Fine_Tuning_model.layers[0].layers[:-11]:
    layer.trainable = False
for layer in Fine_Tuning_model.layers[0].layers[-11:]:
    layer.trainable = True
    
Fine_Tuning_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
Fine_Tuning_model.fit(x_train_preprocessed, y_train, epochs=30, validation_data=(x_test_preprocessed, y_test), callbacks=[WandbCallback()])

Train on 3348 samples, validate on 1440 samples
Epoch 1/30
3348/3348 [==============================] - 52s 15ms/step - loss: 0.2863 - accuracy: 0.9211 - val_loss: 2.5726 - val_accuracy: 0.6132
Epoch 2/30
3348/3348 [==============================] - 47s 14ms/step - loss: 0.0833 - accuracy: 0.9821 - val_loss: 0.5563 - val_accuracy: 0.8549
Epoch 3/30
3348/3348 [==============================] - 47s 14ms/step - loss: 0.0385 - accuracy: 0.9928 - val_loss: 0.5984 - val_accuracy: 0.8521
Epoch 4/30
3348/3348 [==============================] - 47s 14ms/step - loss: 0.0319 - accuracy: 0.9940 - val_loss: 0.6007 - val_accuracy: 0.8618
Epoch 5/30
3348/3348 [==============================] - 47s 14ms/step - loss: 0.0230 - accuracy: 0.9952 - val_loss: 0.6034 - val_accuracy: 0.8556
Epoch 6/30
3348/3348 [==============================] - 47s 14ms/step - loss: 0.0130 - accuracy: 0.9991 - val_loss: 0.5637 - val_accuracy: 0.8701
Epoch 7/30
3348/3348 [==============================] - 47s 14ms/step - loss